# Utiliza o CDB com ML para predizer a localização (lat,lon) das EM:

In [207]:
import pandas as pd
import numpy as np
import math
import os
from tqdm import tqdm
%run Imports.ipynb
%run UtilsLightGBMFingerPrinting.ipynb

In [208]:
#X_test = pd.read_csv("./CDB/CDB_20_SVR_Complete.csv")
#X_test.head()

In [209]:
#X_test = pd.read_csv("./CDB/CDB_20_LightGBM_Complete.csv")
#X_test.head()

In [210]:
FILES = [
    # Arquivos de Teste Outdoor Só possui RSSIs, delays, idx
    (7,'./Test/Outdoor_Test.csv'), # Usar CDB_20_LightGBM_Complete
    (8,'./Test/Indoor_Test.csv'), # Usar CDB_20_LightGBM_Complete
    (9,'./Test/Indoor-Outdoor_Test.csv'), # Usar CDB_20_LightGBM_Complete
    
    #(10,'./Test/Outdoor_Test.csv'), # Usar CDB_20_SVR_Complete
    #(11,'./Test/Indoor_Test.csv'), # Usar CDB_20_SVR_Complete
    #(12,'./Test/Indoor-Outdoor_Test.csv') # Usar CDB_20_SVR_Complete
    
]

TEST_DATABASE = "./CDB/CDB_20_LightGBM_Complete.csv"
#TEST_DATABASE = "./CDB/CDB_20_SVR_Complete.csv"

#TEST_DATABASE = "./CDB/CDB_20_LightGBM_Complete_Med.csv" #Outdoor Melhora, mas o indoor piora
#TEST_DATABASE = "./CDB/CDB_20_SVR_Complete_Med.csv"

In [211]:
def get_nearest_point(test_database, point):
    """
    Encontra o ponto no fingerprint que possui a menor distância euclidiana para o ponto passado como argumento
    """
    min_dist = None
    lat = None
    lon = None
    CDB_Reduced = CDB_ReducedByPDs(point['delay_1'], point['delay_2'], point['delay_3'],
                                   point['delay_12'], point['delay_13'], point['delay_23'], test_database)
    for idx, point_fg in CDB_Reduced.iterrows():
        fg = point_fg[:-2]
        dist = euclidean_distance(point[:-1], fg)
        
        if (min_dist == None) or (min_dist > dist):
            min_dist = dist
            lat = point_fg.lat
            lon = point_fg.lon

    return lat, lon

def calculate_coords_th(fingerprint, test_database):
    """
    Calcula as coordenadas usando um fingerprint e retorna um dataframe contendo
    latitude, longitude e pontoId
    """
    points_dict = {'lat_pred': [], 'lon_pred': [], 'idx': []}
    
    """"
    for idx, point in tqdm(test_database.iterrows()):
        lat, lon = get_nearest_point(fingerprint, point)
        points_dict['lat_pred'].append(lat)
        points_dict['lon_pred'].append(lon)
        points_dict['idx'].append(point.idx)
    """
    for idx, point in tqdm(fingerprint.iterrows()):
        lat, lon = get_nearest_point(test_database, point)
        points_dict['lat_pred'].append(lat)
        points_dict['lon_pred'].append(lon)
        points_dict['idx'].append(point.idx)
    
    
    return pd.DataFrame(points_dict)
    
def generate_theorical_results():
    """
    Calcula as latitudes e longitudes estimadas dos pontos da base de teste usando cada um dos fingerprints
    com as dimensões listadas no vetor DIM
    """
    test_database = pd.read_csv(TEST_DATABASE)
    for method, file in FILES:
        fingerprint = pd.read_csv(file)
        points_df = calculate_coords_th(fingerprint, test_database)
        points_df.to_csv("./FinalResult/Resultados_Metodo_{}.csv".format(method), index=False)

In [212]:
# gera os resultados teoricos
generate_theorical_results()

150it [00:03, 47.70it/s]
150it [00:03, 47.02it/s]
150it [00:03, 46.60it/s]


In [134]:
fingerprint = pd.read_csv('./Test/Outdoor_Test.csv')
for idx, point in fingerprint.iterrows():
    CDB_Reduced = CDB_ReducedByPDs(point['delay_1'], point['delay_2'], point['delay_3'],point['delay_12'], point['delay_13'], point['delay_23'], X_train)
CDB_Reduced.head()

,dist_1,dist_2,dist_3,ang_1,cos_1,sin_1,tg_1,ang_2,cos_2,sin_2,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
50,1.242567,1.001982,0.480684,-171.788725,-0.989748,-0.142824,0.144303,-100.494039,-0.182133,-0.983274,...,79.661828,0.179458,0.983766,5.481882,5,4,2,1,3,2
51,1.190175,0.987275,0.499025,-171.746444,-0.989642,-0.143554,0.145056,-97.596518,-0.132196,-0.991224,...,73.917268,0.277025,0.960863,3.468504,5,4,2,1,3,2
69,1.234103,1.046953,0.436666,-169.449416,-0.983094,-0.183104,0.186252,-99.117843,-0.158466,-0.987365,...,76.376536,0.235540,0.971865,4.126111,5,4,2,1,3,2
114,1.163275,0.979734,0.511192,-171.773682,-0.989711,-0.143084,0.144571,-96.091782,-0.106121,-0.994353,...,71.194579,0.322355,0.946619,2.936570,5,4,2,1,3,2
134,1.234891,1.039690,0.443662,-169.820161,-0.984258,-0.176738,0.179565,-99.305572,-0.161700,-0.986840,...,76.888877,0.226840,0.973932,4.293468,5,4,2,1,3,2


In [45]:
casa = ["rssi_1_1","rssi_1_2","rssi_1_3","rssi_2_1","rssi_2_2","rssi_2_3","rssi_3_1","rssi_3_2","rssi_3_3","delay_1","delay_2","delay_3","delay_12","delay_13","delay_23","idx"]
casa[:-7]

['rssi_1_1',
 'rssi_1_2',
 'rssi_1_3',
 'rssi_2_1',
 'rssi_2_2',
 'rssi_2_3',
 'rssi_3_1',
 'rssi_3_2',
 'rssi_3_3']

In [47]:
casa = ["rssi_1_1","rssi_1_2","rssi_1_3","rssi_2_1","rssi_2_2","rssi_2_3","rssi_3_1","rssi_3_2","rssi_3_3","delay_1","delay_2","delay_3","delay_12","delay_13","delay_23","lat","lon"]
casa[:-8]

['rssi_1_1',
 'rssi_1_2',
 'rssi_1_3',
 'rssi_2_1',
 'rssi_2_2',
 'rssi_2_3',
 'rssi_3_1',
 'rssi_3_2',
 'rssi_3_3']